# Example of different environmental dispatch strategies

In [1]:
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64
!pip install pyomo
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import random 

     |████████████████████████████████| 9.1 MB 8.3 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 


In [2]:
model = AbstractModel()
model.gen=Set()
model.a=Param(model.gen)
model.b=Param(model.gen)
model.c=Param(model.gen)
model.d=Param(model.gen)
model.e=Param(model.gen)
model.f=Param(model.gen)
model.Pmin=Param(model.gen)
model.Pmax=Param(model.gen)

#Scalar
load = 400
Eprice  = 0.1
Elim = 90000

#P.lo(gen)=data(gen,'Pmin')
#P.up(gen)=data(gen,'Pmax')
def Pbounds(model,gen):
  return (model.Pmin[gen],model.Pmax[gen])

model.p=Var(model.gen,bounds=Pbounds,within=Reals)
model.of=Var(within=Reals)
model.TE=Var(bounds=(None,Elim),within=Reals)
model.TC=Var(within=Reals)



def rule1(model,gen):
  return sum(model.p[gen] for gen in model.gen) >= load 
model.c1 = Constraint(model.gen,rule=rule1)

def ruleTC(model,gen):
  return model.TC == sum((model.a[gen]*model.p[gen]*model.p[gen] + 
                              model.b[gen]*model.p[gen]+model.c[gen]) for gen in model.gen) 
model.c2 = Constraint(rule=ruleTC)

def ruleTE(model,gen):
  return model.TE == sum((model.d[gen]*model.p[gen]*model.p[gen] + 
                              model.e[gen]*model.p[gen]+model.f[gen]) for gen in model.gen) 
model.c3 = Constraint(rule=ruleTE)

def ruleof(model,gen):
    return model.of== + model.TC+model.TE * Eprice

model.c4=Constraint(rule=ruleof)

model.obj=Objective(expr=model.of,sense=minimize)

from google.colab import files
  
  
uploaded = files.upload()

instance= model.create_instance("gcode33.dat")
results = SolverFactory('ipopt', executable='/content/ipopt').solve(instance).write()

instance.pprint()
print('OF =   ',value(instance.of))
print('TC =   ',value(instance.TC))
print('TE =   ',value(instance.TE))


Saving gcode33.dat to gcode33.dat
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 8
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.12.13\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.028066396713256836
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  numbe